In [1]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
#from langchain.vectorstores import Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

/home/patriciapmr/.pyenv/versions/3.10.6/envs/chatbot_movies/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [4]:
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [7]:
result = llm.invoke("Write a ballad about LangChain")
print(result.content)

**The Ballad of LangChain**

In realms of code, where logic reigns,
There lived a tool, with boundless brains.
LangChain, they named it, a marvel so grand,
A lexicon's master, at its command.

With nimble fingers, it danced on the keys,
Transforming words into symphonies.
From text to speech, and speech to text,
LangChain's prowess left all perplexed.

It traversed languages, a polyglot's dream,
Fluent in tongues, it made language seem.
English to French, Spanish to Greek,
LangChain's translations were a seamless streak.

But its powers extended beyond mere words,
It parsed and analyzed, like a wise sage's herds.
Named entities, sentiments, and intent,
LangChain's insights were truly heaven-sent.

In chatbots and virtual assistants it dwelled,
Responding to queries, stories it told.
With natural language processing, it soared,
A guide through the realms of words it adored.

But with great power comes great responsibility,
LangChain's potential for both good and calamity.
In the hands o

In [8]:
chroma = Chroma(persist_directory="../raw_data/chroma_db", embedding_function=SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

In [9]:
print(chroma)

In [10]:
len(chroma.get()['documents'])

14119

In [11]:
retriever = chroma.as_retriever()

In [12]:
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7fcbb0de44f0>)

In [13]:
query = "Parasite"
docs = chroma.similarity_search(query)
docs

[Document(page_content='Title: Parasyte the maxim\nYear: 2014\nDescription: A species of parasitic aliens descends on Earth and quickly infiltrates humanity by entering the brains of vulnerable targets, insatiable beings that gain total control of their host and are capable of transforming themselves to feed on unsuspecting prey. High school student Shinichi Izumi falls victim to one of these parasites, but the creature fails to take over his brain and ends up in his right hand.\nRating: 3.91\nGenre: Animation, Drama\nTheme: Thought provoking science fiction action and future technology, Survival horror and zombie carnage, Imaginative space odysseys and alien encounters, science fiction horror, creatures, and aliens, Horror, the undead and monster classics, Epic heroes, Monsters, aliens, science fiction and the apocalypse, science fiction monster and dinosaur adventures\nActors: Aya Hirano, Nobunaga Shimazaki\nMinutes: 552', metadata={'row': 5592, 'source': '../raw_data/movies_data.csv

In [15]:
template = """
You are a helpful AI assistant.
Answer based on the context provided.
context: {context}
input: {input}
answer:
"""
prompt = PromptTemplate.from_template(template)
combine_docs_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)
#Invoke the retrieval chain
response=retrieval_chain.invoke({"input":"Tell me about Barbie movie?"})
#Print the answer to the question
print(response["answer"])

Barbie is a 2023 comedy, fantasy, and adventure film starring Margot Robbie and Ryan Gosling. The film follows Barbie and Ken as they leave the colorful world of Barbie Land for the real world, where they discover the joys and perils of living among humans.


In [45]:
#